In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%pip install transformers datasets

In [18]:
train_path = '/content/drive/MyDrive/SCOTU_data_txt_save'
test_path = '/content/drive/MyDrive/SCOTU_data_txt_save'

train_path_txt = train_path + '/text'
train_path_summary = train_path + '/summary'
test_path_txt = test_path + '/text_dev'
test_path_summary = test_path + '/summary_dev'

In [19]:
import torch

# Vérification de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Entraînement sur : {device}")

Entraînement sur : cuda


In [20]:
import os
from datasets import Dataset

def load_data(txt_path, summary_path):
    """Charge les fichiers texte et résumés pour créer un Dataset."""
    texts = []
    summaries = []

    # Charger les fichiers de texte
    for file_name in os.listdir(txt_path):
        with open(os.path.join(txt_path, file_name), 'r', encoding='utf-8') as f:
            texts.append(f.read())

    # Charger les fichiers de résumés
    for file_name in os.listdir(summary_path):
        with open(os.path.join(summary_path, file_name), 'r', encoding='utf-8') as f:
            summaries.append(f.read())
    # Créer un Dataset Hugging Face
    data = {"text": texts, "summary": summaries}
    return Dataset.from_dict(data)

# Charger les données
train_dataset = load_data(train_path_txt, train_path_summary)
test_dataset = load_data(test_path_txt, test_path_summary)

# Vérifier les exemples
# print(train_dataset[0])

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Charger le modèle et le tokenizer T5
model_name = "google-t5/t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

# Activer le gradient checkpointing pour économiser la mémoire
model.gradient_checkpointing_enable()


In [23]:
def preprocess_function(examples):
    """Tokenisation des entrées et des cibles."""
    inputs = tokenizer(examples["text"], max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(examples["summary"], max_length=256, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

# Préparer les données
train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2670 [00:00<?, ? examples/s]

Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

# Arguments fine-tuning
training_args = TrainingArguments(
    output_dir="./t5-small",
    eval_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    weight_decay=0.01,
    run_name="t5-small-Training",
    report_to=[],
)

# Configuration du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
)

torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

# Lancer l'entraînement
trainer.train()

<ipython-input-24-f75a1bbf0583>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.808100,2.541673
2,2.312200,2.503101
3,2.048800,2.465884


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=8010, training_loss=2.4909885270765213, metrics={'train_runtime': 3660.284, 'train_samples_per_second': 2.188, 'train_steps_per_second': 2.188, 'total_flos': 1.735850786291712e+16, 'train_loss': 2.4909885270765213, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/t5-small-model-Scopus")
tokenizer.save_pretrained("/content/drive/MyDrive/t5-small-model-Scopus")

('/content/drive/MyDrive/BART-model-Scopus/tokenizer_config.json',
 '/content/drive/MyDrive/BART-model-Scopus/special_tokens_map.json',
 '/content/drive/MyDrive/BART-model-Scopus/vocab.json',
 '/content/drive/MyDrive/BART-model-Scopus/merges.txt',
 '/content/drive/MyDrive/BART-model-Scopus/added_tokens.json')